# This project is discovering and clustering Toronto neighborhoods using Foursquare API


# #### This project is part from Coursera IBM Data Science Professional Certificate capstone project 

In [142]:
#import required libraries
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import geopy
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize


In [143]:
#The data I'll work on is part of web page,so I used beautiful soup python library for web scraping
#The data is a table contains Boroughs , Neighborhoods ,Postal Codes of Toronto
result = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [144]:
#check_succesful_webpage_read
print(result.status_code)


200


In [146]:
#read_content
content = result.content


In [148]:
soup = BeautifulSoup(content,'lxml')


In [149]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"cf0149f1-3bc5-409e-b9e6-d59a5559a1a8","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":979555370,"wgRevisionId":979555370,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Communicati

In [150]:
#extract require table from web page
reqtable= soup.find("table",attrs={"class":"wikitable sortable"}).prettify()
print(reqtable)

<table class="wikitable sortable">
 <tbody>
  <tr>
   <th>
    Postal Code
   </th>
   <th>
    Borough
   </th>
   <th>
    Neighbourhood
   </th>
  </tr>
  <tr>
   <td>
    M1A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M2A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M3A
   </td>
   <td>
    North York
   </td>
   <td>
    Parkwoods
   </td>
  </tr>
  <tr>
   <td>
    M4A
   </td>
   <td>
    North York
   </td>
   <td>
    Victoria Village
   </td>
  </tr>
  <tr>
   <td>
    M5A
   </td>
   <td>
    Downtown Toronto
   </td>
   <td>
    Regent Park, Harbourfront
   </td>
  </tr>
  <tr>
   <td>
    M6A
   </td>
   <td>
    North York
   </td>
   <td>
    Lawrence Manor, Lawrence Heights
   </td>
  </tr>
  <tr>
   <td>
    M7A
   </td>
   <td>
    Downtown Toronto
   </td>
   <td>
    Queen's Park, Ontario Provincial Government
   </td>
  </tr>
  <tr>
   <td>
    M8

In [152]:
#assign scraped table data into a list
data=[]

tablerows = soup.find_all('tr')

for row in tablerows:
    tablecells = row.find_all("td")
    cells=[d.text.strip() for d in tablecells if d.text.strip()]
    if cells:
        data.append(cells)

del data[-1]
del data[-2]
del data[-3]
del data[-4]
del data[-5]
data

    


[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'],
 ['M6A', 'North York', 'Lawrence Manor, Lawrence Heights'],
 ['M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', 'Etobicoke', 'Islington Avenue, Humber Valley Village'],
 ['M1B', 'Scarborough', 'Malvern, Rouge'],
 ['M2B', 'Not assigned', 'Not assigned'],
 ['M3B', 'North York', 'Don Mills'],
 ['M4B', 'East York', 'Parkview Hill, Woodbine Gardens'],
 ['M5B', 'Downtown Toronto', 'Garden District, Ryerson'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', 'Not assigned'],
 ['M8B', 'Not assigned', 'Not assigned'],
 ['M9B',
  'Etobicoke',
  'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale'],
 ['M1C', 'Scarborough', 'Rouge Hill, Port Union, Highland Creek'],
 ['M

In [153]:
#convert list into dataframe
df = pd.DataFrame(data[:-1] , columns=["Postalcode" , "Borough" , "Neighborhood"])
df.head(10)


,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [154]:
#drop not assigned boroughs
df.drop(df.loc[df['Borough']=='Not assigned'].index , inplace=True)
df.head(10)

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [54]:
df.shape

(102, 3)

In [55]:
df.reset_index(drop=True, inplace=True)
df.head(10)

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [56]:
df.nunique()

Postalcode      102
Borough          10
Neighborhood     98
dtype: int64

In [57]:
df.dtypes

Postalcode      object
Borough         object
Neighborhood    object
dtype: object

In [60]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [61]:
#geocoder is useful library used with geographical data
import geocoder

In [ ]:
#By geocoder we can get latitude and longitude from postal code by this code but sometimes it's not reliable so I get them from available csv file direct instead
""""
def get_geocoder(pc):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{},Toronto,Ontario'.format(pc))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude , longitude



#df['Longitude']  = zip(*df['Postalcode'].apply(get_geocoder))

for i in range(0,len(df)):
    df.iloc[i]['Latitude'],df.iloc[i]['Longitude'] =get_geocoder(df.iloc[i]['Postalcode'])
"""
    

In [155]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [27]:
# The code was removed by Watson Studio for sharing.

In [156]:
#I read the lat,lng data as coords dataframe and joined it with original dataframe df
data = df.join(coords.set_index('Postal Code'), on = 'Postalcode' )
data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [157]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(data['Borough'].unique()),
        data.shape[0]
    )
)

The dataframe has 10 boroughs and 102 neighborhoods.


In [65]:
#geopy is useful library you can use in case you want to get latitude,longitude of a place from it's address
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [158]:
#we have got the lat , lng of city Toronto by using geopy

address = 'Toronto , Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude , longitude)




43.6534817 -79.3839347


In [67]:
#folium is useful library for visualizing
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [159]:
#We drew a map for Toronto with it's neighborhoods marked on it by folium
import folium

Toronto_map = folium.Map(location=[latitude,longitude] , zoom_start=11)

for lat,lng,label in zip(data['Latitude'] , data['Longitude'] , data['Neighborhood']):
    label = folium.Popup(label , parse_html=True)
    folium.CircleMarker([lat,lng] , radius=5 , popup=label , color='blue' , fill=True , fill_color='#3186cc',fill_opacity=0.7, parse_html=False).add_to(Toronto_map)
    
Toronto_map


In [160]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: HITSIQVW0SB5FAPB1MUXLQPZJKTNO3A0IOBMQ2XMJYPHBE3W
CLIENT_SECRET:B0TCEFOOI22OZNFINSXLA2NHK3KZO5SMPHEWXUKHZQAOLR33


### Now, I'll explore nearby venues for only first neighborhood in the data

In [ ]:
# At first I'll explore the venues in the first neighborhood only in our data, so we extracted it's name,lng,lat
N_latitude = data.loc[0 , 'Latitude']
N_longitude = data.loc[0 , 'Longitude']
N_name = data.loc[0 , 'Neighborhood']



In [70]:
#I used foursquare api for the exploring, so below is the url for access your personal developer account and venues exploring
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, N_latitude, N_longitude,ACCESS_TOKEN, VERSION, radius, LIMIT)
url
    
    


'https://api.foursquare.com/v2/venues/explore?client_id=HITSIQVW0SB5FAPB1MUXLQPZJKTNO3A0IOBMQ2XMJYPHBE3W&client_secret=B0TCEFOOI22OZNFINSXLA2NHK3KZO5SMPHEWXUKHZQAOLR33&ll=43.7532586,-79.3296565&oauth_token=QIGWBEUZCXNOCZYSH5ROWUXXJWLS53BMT0RRO1HMMM3XZ2NP&v=20180605&radius=500&limit=100'

In [71]:
#request api to get output target venues in json file 
result = requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '5fdbdaafe3f54d3244a89efd'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 10,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.751976046055574,
   

In [72]:
#Select only target content from json file which is explored venues
venues = result['response']['groups'][0]['items'][0]['venue']

#Then transform it from json to dataframe
dataframe = json_normalize(venues)
dataframe.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,id,name,categories,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,photos.count,photos.groups,venuePage.id
0,4e8d9dcdd5fbbbb6b3003c7b,Brookbanks Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",Toronto,43.751976,-79.33214,"[{'label': 'display', 'lat': 43.75197604605557...",245,CA,Toronto,ON,Canada,"[Toronto, Toronto ON, Canada]",0,[],600917367


In [73]:
# function that extracts the category of the venue from jsonfile
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [74]:
f#ilter exploring venues dataframe for only interested columns to simplify it
filtered_columns= ["name" , "categories" , "location.lat" , "location.lng" ]
dataframe = dataframe.loc[: , filtered_columns]
dataframe

,name,categories,location.lat,location.lng
0,Brookbanks Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",43.751976,-79.33214


In [75]:
#add venue category to dataframe
dataframe['category'] = dataframe.apply(get_category_type , axis=1)
dataframe

,name,categories,location.lat,location.lng,category
0,Brookbanks Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",43.751976,-79.33214,Park


### I'll repeat the above exploring but this time for all the neighborhoods instead of only one

In [111]:
#function for exploring all neighborhoods and add the important explored venues data with original neighborhoods data in one data frame 
def get_nearby_venues(names,latitudes,longitudes,radius=500):
    
    venue_list=[]
    for name,lat,lng in zip(names,latitudes,longitudes):
        #print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&oauth_token={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            ACCESS_TOKEN,
            LIMIT)
        
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        venue_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng']) for v in results])
            
    nearby_venues = pd.DataFrame([item for venue_list in venue_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                'Neighborhood Latitude', 
                'Neighborhood Longitude', 
                'Venue', 
                'Venue Latitude', 
                'Venue Longitude']
                  
        
        
    return(nearby_venues)

In [113]:
#call function and pass original data to work on
Toronto_venues = get_nearby_venues(names=data['Neighborhood'] , latitudes=data['Latitude'], longitudes=data['Longitude'])



In [114]:
#final data after exploring
Toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719
1,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741
2,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368
3,Parkwoods,43.753259,-79.329656,Galleria Supermarket,43.753520,-79.349518
4,Parkwoods,43.753259,-79.329656,Island Foods,43.745866,-79.346035


In [97]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Neighborhood,,,,,
Agincourt,100,100,100,100,100
"Alderwood, Long Branch",100,100,100,100,100
"Bathurst Manor, Wilson Heights, Downsview North",100,100,100,100,100
Bayview Village,100,100,100,100,100
"Bedford Park, Lawrence Manor East",100,100,100,100,100
...,...,...,...,...,...
"Willowdale, Willowdale West",100,100,100,100,100
Woburn,100,100,100,100,100
Woodbine Heights,100,100,100,100,100


# ## Let's start analyzing the output data and prepare it for clustering

In [122]:
# one hot encoding
toronto_onehot = pd.get_dummies(Toronto_venues[['Venue']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,0109 Dessert & Chocolate,120 Diner,17 Steak House and Bar,3030 Dundas,306 Yonge Street - Jordan Store,360 Restaurant,4 Life Natural Foods,401 Diner,401 Games,...,nodo,nutbar,the sushi bar,touti gelati and café,west coast kids,½ oz. Cocktail Emporium,Étienne Brulé Park,九鼎香 Legendary Hot Pot,天天渔港 Captain's Catch,고려삼계탕 Korean Ginseng Chicken Soup & Bibimbap
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [126]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,0109 Dessert & Chocolate,120 Diner,17 Steak House and Bar,3030 Dundas,306 Yonge Street - Jordan Store,360 Restaurant,4 Life Natural Foods,401 Diner,401 Games,...,nodo,nutbar,the sushi bar,touti gelati and café,west coast kids,½ oz. Cocktail Emporium,Étienne Brulé Park,九鼎香 Legendary Hot Pot,天天渔港 Captain's Catch,고려삼계탕 Korean Ginseng Chicken Soup & Bibimbap
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.01
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
94,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
95,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
96,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.00


In [128]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                                venue  freq
0                  Shoppers Drug Mart  0.03
1                         Tim Hortons  0.03
2                              Subway  0.02
3     Best Western Plus Executive Inn  0.01
4  The Royal Chinese Restaurant 避風塘小炒  0.01


----Alderwood, Long Branch----
          venue  freq
0     Starbucks  0.03
1          LCBO  0.02
2  Burrito Boyz  0.02
3     Five Guys  0.02
4    Kumo Sushi  0.01


----Bathurst Manor, Wilson Heights, Downsview North----
                venue  freq
0           Starbucks  0.05
1  Aroma Espresso Bar  0.03
2         Tim Hortons  0.02
3         Dairy Queen  0.02
4     TD Canada Trust  0.01


----Bayview Village----
          venue  freq
0     Starbucks  0.02
1          LCBO  0.02
2       Loblaws  0.02
3           GAP  0.01
4  Maryam Hotel  0.01


----Bedford Park, Lawrence Manor East----
                venue  freq
0           Starbucks  0.05
1  Aroma Espresso Bar  0.03
2           The Abbot  0.02
3   Grodzinsk

In [129]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [131]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Shoppers Drug Mart,Tim Hortons,Subway,Mona's Roti,Real Canadian Superstore,Fairview Seafood Chinese Cuisine,Wonton Chai Noodle 雲吞仔,Coco Fresh Tea And Juice,Disney Store,The Royal Chinese Restaurant 避風塘小炒
1,"Alderwood, Long Branch",Starbucks,LCBO,Burrito Boyz,Five Guys,Sweet Olenka's,Beaumont Kitchen,Costco,JOEY Sherway,Moksha Yoga Etobicoke,Istanbul Kebab House
2,"Bathurst Manor, Wilson Heights, Downsview North",Starbucks,Aroma Espresso Bar,Tim Hortons,Dairy Queen,Grandprix Kartways,Archers Arena,Apple Yorkdale,Bake Code 烘焙密碼,Konjiki Ramen,Wolfie's Deli
3,Bayview Village,LCBO,Starbucks,Loblaws,고려삼계탕 Korean Ginseng Chicken Soup & Bibimbap,Hub Sushi,Maryam Hotel,Mastermind Toys,Bow Thai,Nino D'Aversa,Hot Spicy Spicy Chinese Restaurant 麻辣烫川菜馆
4,"Bedford Park, Lawrence Manor East",Starbucks,Aroma Espresso Bar,The Abbot,Li'l Niblets & Baby Sprouts,Mastermind Toys,Ritché Bridal,Auberge du Pommier,GoodLife Fitness North York York Mills Centre,De Mello Palheta Coffee Roasters,Riz North


# ## Let's cluster the venues output data by using K-Means Cluster Algorithm

In [132]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:60] 

array([2, 4, 4, 4, 4, 0, 1, 3, 3, 0, 3, 2, 2, 4, 3, 4, 2, 1, 0, 4, 4, 3,
       1, 2, 2, 3, 1, 4, 2, 0, 4, 0, 4, 1, 2, 0, 3, 2, 2, 2, 3, 1, 4, 1,
       3, 2, 4, 4, 4, 3, 2, 2, 4, 4, 4, 4, 2, 2, 4, 3], dtype=int32)